In [1]:
# Import stuff

import re
from BCBio import GFF
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
import pysam

## Functions

In [2]:
def extract_genes():
    '''
    Function for extracting genes corresponding to BUSCO hits.
    Returns a SeqRecord object with one feature per BUSCO hit.
    '''
    # TODO: File path based on run name, take as input to the function
    file_tsv = open("../run_4_gen/full_table_4.tsv", 'r')

    # Extract BUSCO IDs, start and end from table of hits into SeqRecord object, each BUSCO as a SeqFeature
    busco_record = SeqRecord(seq='', id='hits')
    for line in file_tsv.readlines():
        hit = (re.search(r'(\S*)\sComplete\s(\S*)\s(\S*)\s(\S*)\s\S*', line))
        if hit:
            busco_record.features.append(SeqFeature(FeatureLocation(int(hit.group(3)), int(hit.group(4))), id=hit.group(1), type='gene', qualifiers={'contig': hit.group(2)}))

    file_tsv.close()

    # Match the BUSCOs to augustus predicted genes in gff file
    gff_records = []
    correct_genes = SeqRecord(seq='', id='correct_genes')
    limit_infos = dict(
            gff_type = ["gene"]) # Only want genes

    for busco in busco_record.features:
        filename = busco.id # gff filenames are {busco_id}.out.1
        i = 1
        while True:
            try:
                file_gff = open("../run_4_gen/augustus_output/predicted_genes/"+filename+".out."+str(i))
                for record in GFF.parse(file_gff, limit_info=limit_infos):
                    gff_records.append(record)
                i += 1
            except: # read all files for that BUSCO
                break

    # Find augustus predicted genes from the gff that match BUSCOs
    for rec in gff_records:
        for hit in busco_record.features:
            for feature in rec.features:
                if hit.location.start-1 == feature.location.start and hit.location.end == feature.location.end: # For some reason start has 1 nt diff...
                    #feature.qualifiers = {'contig': rec.id}
                    feature.id = rec.id # save as qualifier or id?
                    correct_genes.features.append(feature)
                    break

    file_gff.close()
    return correct_genes

In [3]:
def extract_mapped():
    '''
    Function used to extract only reads mapped to a reference.
    Really slow, only used to minimize the reads dataset
    '''
    pairedreads = pysam.AlignmentFile("allpaired2.bam", "wb", template=samfile)
    mapped_reads =[]

    for read in samfile.fetch():
        # Probably stupidly long if-statement
        if read.is_read1 and not read.is_secondary and not read.is_unmapped and not read.mate_is_unmapped and read not in mapped_reads:
            mapped_reads.append(read)
            pairedreads.write(read)
            print(read)
            print(samfile.mate(read))
            pairedreads.write(samfile.mate(read))

    return mapped_reads

In [10]:
def infer_region(contig, start, stop, strand):
    '''
    Function inferring lib-type of a specific region
    '''
    reads = []
    # Get reads mapped to a specific contig and in a sequence range
    for read in samfile.fetch(contig, start, stop):
        if not read.mate_is_unmapped and read.is_read1:
            reads.append([read, samfile.mate(read)])

    # Counters for the different lib-types   
    libs = {
        'fr_first': 0,
        'fr_second': 0,
        'rf_first': 0,
        'rf_second': 0,
        'ff_first': 0,
        'ff_second': 0,
        'undecided': 0
    }
    
    if strand == 1: # Gene on plus strand
        # Check lib-type of reads 
        # TODO: check if the reads are in-or outfacing
        for read in reads:
            first = read[0]
            second = read[1]
            try:
                lib = ''
                if not first.is_reverse:
                    lib += 'f'
                else: 
                    lib += 'r'
                if not second.is_reverse:
                    lib += 'f'
                else: 
                    lib += 'r'
                # TODO: Make sure firststrand and secondstrand are correctly assigned
                if first.reference_start > second.reference_start:
                    lib = lib[::-1]
                    lib += '_first'
                elif first.reference_start < second.reference_start:
                    lib += '_second'
                else:
                    undecided +=1
                libs[lib] += 1
            except: libs['undecided'] +=1 # Some reads missing start or end-values
    #elif strand == -1: # Gene on minus strand
        # TODO: reverse counting

    
    return [reads, libs]

## Running

In [5]:
# bam-file with mapped reads
samfile = pysam.AlignmentFile("../bowtie_test/fly_sorted.bam", "rb")

In [6]:
test = extract_genes()

In [7]:
print(test)

ID: correct_genes
Name: <unknown name>
Description: <unknown description>
Number of features: 2
''


In [8]:
test.features[0].strand

1

In [12]:
# TODO: Implement get_location for contig, start and end from each gene
[reads, libs] = infer_region('4', 155151, 164760, 1)


In [13]:
print(libs)

{'fr_first': 271, 'fr_second': 318, 'rf_first': 3, 'rf_second': 0, 'ff_first': 0, 'ff_second': 0, 'undecided': 46}


Seems correct, found one of the rf-first when quickly looking at the mapping in IGV. 